## Deliverable 2. Create a Customer Travel Destinations Map.

In [22]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [11]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/linghoang/UCBBootcamp/DataClass/Module6/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Qaanaaq,GL,77.4840,-69.3632,42.26,65,38,5.97,scattered clouds
1,1,Cayenne,GF,4.9333,-52.3333,79.09,1,0,3.44,clear sky
2,2,West Wendover,US,40.7391,-114.0733,78.01,41,20,5.75,few clouds
3,3,Butaritari,KI,3.0707,172.7902,82.09,73,89,19.98,overcast clouds
4,4,Coihaique,CL,-45.5752,-72.0662,30.81,93,0,3.44,clear sky


In [12]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 86


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Cayenne,GF,4.9333,-52.3333,79.09,1,0,3.44,clear sky
2,2,West Wendover,US,40.7391,-114.0733,78.01,41,20,5.75,few clouds
3,3,Butaritari,KI,3.0707,172.7902,82.09,73,89,19.98,overcast clouds
7,7,Boende,CD,-0.2167,20.8667,71.40,86,99,1.97,overcast clouds
10,10,Jamestown,US,42.0970,-79.2353,69.39,75,75,11.50,moderate rain
11,11,Didsbury,GB,53.4170,-2.2315,67.80,95,75,9.22,heavy intensity rain
14,14,Rikitea,PF,-23.1203,-134.9692,73.27,74,24,24.70,few clouds
15,15,Kibala,TD,9.1104,18.3463,70.09,98,100,4.29,light rain
17,17,Praia,CV,14.9215,-23.5087,73.94,83,20,4.61,few clouds
21,21,Hilo,US,19.7297,-155.0900,75.27,84,40,4.00,light rain


In [14]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             397
City                   397
Country                397
Lat                    397
Lng                    397
Max Temp               397
Humidity               397
Cloudiness             397
Wind Speed             397
Current Description    397
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
print(clean_df)

     Unnamed: 0              City Country      Lat       Lng  Max Temp  \
1             1           Cayenne      GF   4.9333  -52.3333     79.09   
2             2     West Wendover      US  40.7391 -114.0733     78.01   
3             3        Butaritari      KI   3.0707  172.7902     82.09   
7             7            Boende      CD  -0.2167   20.8667     71.40   
10           10         Jamestown      US  42.0970  -79.2353     69.39   
..          ...               ...     ...      ...       ...       ...   
701         701  Kuala Terengganu      MY   5.3302  103.1408     84.15   
702         702           Cordele      US  31.9635  -83.7824     77.52   
703         703         Kontagora      NG  10.3999    5.4695     69.03   
707         707           Sambava      MG -14.2667   50.1667     72.66   
709         709           Salalah      OM  17.0151   54.0924     75.29   

     Humidity  Cloudiness  Wind Speed      Current Description  
1           1           0        3.44         

In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Cayenne,GF,79.09,clear sky,4.9333,-52.3333,
2,West Wendover,US,78.01,few clouds,40.7391,-114.0733,
3,Butaritari,KI,82.09,overcast clouds,3.0707,172.7902,
7,Boende,CD,71.40,overcast clouds,-0.2167,20.8667,
10,Jamestown,US,69.39,moderate rain,42.0970,-79.2353,
11,Didsbury,GB,67.80,heavy intensity rain,53.4170,-2.2315,
14,Rikitea,PF,73.27,few clouds,-23.1203,-134.9692,
15,Kibala,TD,70.09,light rain,9.1104,18.3463,
17,Praia,CV,73.94,few clouds,14.9215,-23.5087,
21,Hilo,US,75.27,light rain,19.7297,-155.0900,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [19]:
len(hotel_df)

397

In [20]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Cayenne,GF,79.09,clear sky,4.9333,-52.3333,Hôtel Le Dronmi
2,West Wendover,US,78.01,few clouds,40.7391,-114.0733,"Motel 6 Wendover, UT"
3,Butaritari,KI,82.09,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
7,Boende,CD,71.40,overcast clouds,-0.2167,20.8667,
10,Jamestown,US,69.39,moderate rain,42.0970,-79.2353,Hampton Inn & Suites Jamestown
11,Didsbury,GB,67.80,heavy intensity rain,53.4170,-2.2315,Eleven Didsbury Park
14,Rikitea,PF,73.27,few clouds,-23.1203,-134.9692,People ThankYou
15,Kibala,TD,70.09,light rain,9.1104,18.3463,Prasant Hotels
17,Praia,CV,73.94,few clouds,14.9215,-23.5087,LT Aparthotel
21,Hilo,US,75.27,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel


In [23]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Cayenne,GF,79.09,clear sky,4.9333,-52.3333,Hôtel Le Dronmi
2,West Wendover,US,78.01,few clouds,40.7391,-114.0733,"Motel 6 Wendover, UT"
3,Butaritari,KI,82.09,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
10,Jamestown,US,69.39,moderate rain,42.0970,-79.2353,Hampton Inn & Suites Jamestown
11,Didsbury,GB,67.80,heavy intensity rain,53.4170,-2.2315,Eleven Didsbury Park
14,Rikitea,PF,73.27,few clouds,-23.1203,-134.9692,People ThankYou
15,Kibala,TD,70.09,light rain,9.1104,18.3463,Prasant Hotels
17,Praia,CV,73.94,few clouds,14.9215,-23.5087,LT Aparthotel
21,Hilo,US,75.27,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
24,Nome,US,67.15,broken clouds,64.5011,-165.4064,Dredge 7 Inn


In [24]:
# 8a. Create the output File (CSV)
output_data_file = "/Users/linghoang/UCBBootcamp/DataClass/Module6/World_Weather_Analysis/Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [28]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig



Figure(layout=FigureLayout(height='420px'))